In [141]:
import sklearn
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [142]:
# Import the data
tree = pd.read_csv('tree.csv')
tree['t'] = tree['t'].replace(to_replace=0, value=0.1)

vert_genes = pd.read_csv('vert_genes.csv')

## Part I: Simulation

In [143]:
# Creating the graph

def create_graph(tree, alpha, beta, sigma_sq):
    G = nx.DiGraph()
    for _, row in tree.iterrows():
        if not pd.isna(row['Parent']):
            G.add_edge(int(row["Parent"]), int(row["Child"]), time = row["t"], a = alpha*row["t"], b = beta, variance = sigma_sq*row["t"])
            
    return G

G = create_graph(tree, alpha = 0, beta = 1, sigma_sq = 2500)

In [144]:
def simulate_node_length_with_parameters(G, parent, simulated_lengths, alpha, beta, sigma_sq):
    for child in G.successors(parent):
        t = G[parent][child]['time']
        mean = alpha * t + beta * simulated_lengths[parent]
        std = np.sqrt(sigma_sq * t)
        simulated_lengths[child] = np.random.normal(mean, std)
        simulate_node_length_with_parameters(G, child, simulated_lengths, alpha, beta, sigma_sq)
    
    return simulated_lengths

In [145]:
def simulate_data_for_learning(G, n, alpha, beta, sigma_sq, alpha_0, sigma_0_sq, root, learn_params = True, only_X = True):
    X_values = []
    Y_values = []
    
    all_nodes = list(G.nodes)  # Get all nodes in the graph

    for _ in range(n):
        simulated_lengths = {}
        
        # Simulate root node first
        simulated_lengths[root] = np.random.normal(alpha_0, np.sqrt(sigma_0_sq))
        
        # Simulate all other nodes recursively
        simulated_lengths = simulate_node_length_with_parameters(G, root, simulated_lengths, alpha, beta, sigma_sq)
        if only_X:
            leaf_nodes = [node for node in all_nodes if G.out_degree(node) == 0]
            simulated_x = [simulated_lengths[node] for node in leaf_nodes]
            
            X_values.append(simulated_x)
        else:
            X_values.append([simulated_lengths[node] for node in all_nodes])
            
        if learn_params:
            Y_values.append([alpha,beta,sigma_sq])
        else:
            Y_values.append(simulated_lengths[root])

    return np.array(X_values), np.array(Y_values)

learn_parameters = False

X, y = simulate_data_for_learning(G, 1000, alpha = 0.5, beta = 1, sigma_sq = 2500, alpha_0 = 50000, sigma_0_sq = 5000, root = 407, learn_params=learn_parameters)

print(X.shape)
print(y.shape)

print(y[10])

(1000, 204)
(1000,)
49930.60234858928


In [146]:
def compute_gamma(G):
    gamma = np.eye(len(G.nodes)) # Initialize gamma as an identity matrix

    # Iterate through the nodes in the graph
    for node in G.nodes:
        parent = next(G.predecessors(node), None)
        if parent is None:
            continue
        else:
            gamma[parent-1, node -1] = -G[parent][node]['b'] # Determine the dependency between parent and child nodes with -b
    return gamma

def compute_beta(G, alpha_0):
    beta = np.zeros((len(G.nodes), 1))
    for node in G.nodes:
        parent = next(G.predecessors(node), None)
        if parent is None:
            beta[node-1] = alpha_0
            continue
        a = G[parent][node]['a'] # The constant term in the mean of the CPD
        beta[node-1] = a
        
    return beta

def compute_sigma(G, sigma_0_sq):
    sigma = np.zeros((len(G.nodes)))
    for node in G.nodes:
        parent = next(G.predecessors(node), None)
        if parent is None:
            # Assign the default value for the root node
            sigma[node - 1] = sigma_0_sq
        else:
            # Access the edge attribute 'variance' only if parent exists
            variance = G[parent][node]['variance']
            sigma[node - 1] = variance
    return sigma

def compute_J_and_h(alpha, beta, sigma_sq, sigma_0_sq, alpha_0):
    
    G = create_graph(tree, alpha, beta, sigma_sq)

    beta = compute_beta(G, alpha_0)
    sigma = compute_sigma(G, sigma_0_sq)
    gamma = compute_gamma(G)

    J = np.sum([np.outer(gamma[:, i], gamma[:, i]) / sigma[i] for i in range(len(G))], axis=0)
    h = np.sum([(beta[i] / sigma[i]) * gamma[:, i] for i in range(len(G))], axis=0)
    return J, h



In [147]:
n = 1000

alpha = 0
beta = 1
sigma_sq = 2500
All_nodes_simulated, _ = simulate_data_for_learning(G, n, alpha=alpha, beta=beta, sigma_sq=sigma_sq, alpha_0=50000, sigma_0_sq=5000, root=407, learn_params=False, only_X=False)
empirical_covariance_matrix = np.cov(All_nodes_simulated, rowvar=False)

J, h = compute_J_and_h(alpha = alpha, beta = beta, sigma_sq = sigma_sq, sigma_0_sq=5000, alpha_0=50000)

computed_covariance_matrix = np.linalg.inv(J)

print(empirical_covariance_matrix.shape, computed_covariance_matrix.shape)


difference =  empirical_covariance_matrix - computed_covariance_matrix
print(f"Difference between empirical and computed covariance matrix \n: {difference}")

h

(407, 407) (407, 407)
Difference between empirical and computed covariance matrix 
: [[ -23598.74283951   -2077.61413687    7239.68642705 ...    3916.47944992
     4297.87589844    5147.61419699]
 [  -2077.61413687   -3402.99201237    8230.37400879 ...    1906.74810246
     3401.22724416    4129.24387502]
 [   7239.68642705    8230.37400879    1275.1482611  ...    6108.09979759
     3168.41440028    3921.71192039]
 ...
 [   3916.47944992    1906.74810246    6108.09979759 ...   -7325.80000603
  -104014.36528349    2440.75939477]
 [   4297.87589844    3401.22724416    3168.41440028 ... -104014.36528349
   -20057.65056599   93384.38969634]
 [   5147.61419699    4129.24387502    3921.71192039 ...    2440.75939477
    93384.38969634   98439.23639781]]


array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0

In [148]:
def compute_clique_tree(G):
    C = nx.Graph()

    G_working = G.copy()
    leaves = [node for node in G_working.nodes if G_working.out_degree(node) == 0]
    G_working.remove_nodes_from(leaves)

    index = min(G_working.nodes) -1 if all(isinstance(n, int) for n in G_working.nodes) else 0

    for node in G_working.nodes:
        parent = node
        children = list(G_working.neighbors(parent))
        C.add_node(parent, variables=[parent])
        for child in children:
            pair_clique = index
            C.add_node(pair_clique, variables=[parent, child])
            C.add_edge(parent, pair_clique)
            C.add_edge(pair_clique, child)
            index = index - 1

    return C

C = compute_clique_tree(G)

In [149]:
NoV = len([node for node in C.nodes if len(C.nodes[node]['variables']) == 1]) -1
maxIndex = max([node for node in C.nodes])
away_from_zero = maxIndex - NoV
minIndex = min([node for node in C.nodes if len(C.nodes[node]['variables']) == 1])

print("Away from zero: ", away_from_zero)
print("Max index: ", maxIndex)
print("Min index: ", minIndex)
print("Number of variables in the graph: ", NoV)

def mapping_GTM(index_in_graph):
    return index_in_graph - away_from_zero 

def mapping_MTG(index_in_matrix):
    return index_in_matrix + away_from_zero

Away from zero:  205
Max index:  407
Min index:  205
Number of variables in the graph:  202


### Matrix algebra implementation

In [150]:
def get_sub_matrices(scope, X_values, J, h):
    X_indices = np.isin(scope, X_values)
    Z_indices = ~X_indices

    J_ZZ = J[Z_indices, :][:, Z_indices]
    J_ZX = J[Z_indices, :][:, X_indices]
    J_XZ = J_ZX.T
    J_XX = J[X_indices, :][:, X_indices]
    J_ZZ_inv = np.linalg.inv(J_ZZ)

    h_X = h[X_indices]
    h_Z = h[Z_indices]

    return J_ZZ, J_ZX, J_XZ, J_XX, J_ZZ_inv, h_X, h_Z

def get_conditional_distribution(J, h, X_values, X_indices):
    scope = [i for i in range(1, len(J) + 1)]
    J_ZZ, J_ZX, J_XZ, J_XX, J_ZZ_inv, h_X, h_Z = get_sub_matrices(scope, X_indices, J, h)

    J_reduced = J_ZZ
    h_reduced = h_Z- J_ZX @ X_values

    return J_reduced, h_reduced

X_values = simulate_data_for_learning(G, 1, alpha=alpha, beta=beta, sigma_sq=sigma_sq, alpha_0=50000, sigma_0_sq=5000, root=407, learn_params=False, only_X=True)[0]


leaves = [node for node in G.nodes if G.out_degree(node) == 0]
X_indices = leaves

J_reduced, h_reduced = get_conditional_distribution(J, h, X_values[0], X_indices)

In [ ]:
Sigma = np.linalg.inv(J_reduced)
mu = Sigma @ h_reduced

random_index = np.random.choice(range(len(X_values[0])))
z = mapping_GTM(random_index)
print(f"Predicted value for node 407: {mu[z]}")
print(f"Variance for node 407: {Sigma[z,z]}")
print(f"Actual value for node 407: {X_values[0][0]}")
print(f"True variance for node 407: {sigma_sq}")

Predicted value for node 407: 49792.31898955036
Variance for node 407: 5926.1487494450475
Actual value for node 407: 49398.97361219418
True variance for node 407: 2500


## Part II : inference

In [168]:
def single_clique(G):
    leaves = [node for node in G.nodes if G.out_degree(node) == 0]
    H = G.copy()
    H.remove_nodes_from(leaves)
    H = H.to_undirected()
    return H

def compute_J_i_arrow_j(clique_tree, i, j, J, h, J_messages, h_messages):
    
    neighbors = list(clique_tree.neighbors(i))
    neighbors.remove(j)

    i_idx = mapping_GTM(i)
    j_idx = mapping_GTM(j)

    if not neighbors:
        J_messages[i_idx][j_idx] = -J[i_idx, j_idx] * J[j_idx, i_idx] / J[i_idx, i_idx]
    else:
        J_sum = sum(compute_J_i_arrow_j(clique_tree, k, i, J, h, J_messages, h_messages) for k in neighbors)
        J_messages[i_idx][j_idx] = -J[i_idx, j_idx] * J[j_idx, i_idx] / (J[i_idx, i_idx] + J_sum)

    return J_messages[i_idx][j_idx]


def compute_h_i_arrow_j(clique_tree, i, j, J, h, J_messages, h_messages):

    neighbors = list(clique_tree.neighbors(i))
    neighbors.remove(j)

    i_idx = mapping_GTM(i)
    j_idx = mapping_GTM(j)

    if not neighbors:
        h_messages[i_idx][j_idx] = -J[i_idx, j_idx] * h[i_idx] / J[i_idx, i_idx]
    else:
        J_sum = sum(compute_J_i_arrow_j(clique_tree, k, i, J, h, J_messages, h_messages) for k in neighbors)
        h_sum = sum(compute_h_i_arrow_j(clique_tree, k, i, J, h, J_messages, h_messages) for k in neighbors)
        Ji_backslash_j = J[i_idx, i_idx] + J_sum
        hi_backslash_j = h[i_idx] + h_sum
        h_messages[i_idx][j_idx] = (-J[i_idx, j_idx] * hi_backslash_j) / (Ji_backslash_j)

    return h_messages[i_idx][j_idx]

def inference_algorithm(G, alpha, beta, sigma_sq, alpha_0, observed_X, Z):

    clique_tree = single_clique(G)
    J, h = compute_J_and_h(alpha, beta, sigma_sq, 5000, alpha_0)

    n_observed = len(observed_X)
    J_reduced, h_reduced = get_conditional_distribution(J, h, observed_X, list(range(1, n_observed + 1)))

    J_messages = np.full(J_reduced.shape, np.nan)
    h_messages = np.full(J_reduced.shape, np.nan)

    Z_neighbors = list(clique_tree.neighbors(Z))
    Z_idx = mapping_GTM(Z)

    J_zz = J_reduced[Z_idx, Z_idx]

    sum_J = sum([compute_J_i_arrow_j(clique_tree, k, Z, J_reduced, h_reduced, J_messages, h_messages) for k in Z_neighbors])
    sum_h = sum([compute_h_i_arrow_j(clique_tree, k, Z, J_reduced, h_reduced, J_messages, h_messages) for k in Z_neighbors])

    J_hat_Z = J_zz + sum_J
    h_hat_Z = h_reduced[Z_idx] + sum_h


    return J_hat_Z, h_hat_Z

def information_to_standard(J_hat_Z, h_hat_Z, Z):

    mu = h_hat_Z / J_hat_Z
    sigma = np.sqrt(1 / J_hat_Z)
    return mu, sigma

n = 10
random_index = np.random.choice(range(n))
Z = 407  # root node

alpha = 0
beta = 1
sigma_sq = 2500
alpha_0 = 50000

X_values, _ = simulate_data_for_learning(G, n, alpha = alpha, beta = beta, sigma_sq = sigma_sq, alpha_0 = 50000, sigma_0_sq = 5000, root = 407, learn_params=False, only_X=True)


J_hat_Z, h_hat_Z = inference_algorithm(G, alpha, beta, sigma_sq, alpha_0, X_values[random_index], Z)
mu, sigma = information_to_standard(J_hat_Z, h_hat_Z, Z)

true_mu = X_values[random_index][0]
true_sigma = sigma_sq
print("True mean:", true_mu)
print("True std dev:", true_sigma)
print("Posterior mean:", mu)
print("Posterior std dev:", sigma)

True mean: 49560.2929693672
True std dev: 2500
Posterior mean: 49950.05573773816
Posterior std dev: 60.085211992236054


## Part III: Learning

In [160]:
def simulate_full_data(G, n_samples, alpha, beta, sigma_sq, alpha_0, sigma_0_sq):
    rows = []

    for _ in range(n_samples):
        simulated = {}
        simulated[root] = np.random.normal(alpha_0, np.sqrt(sigma_0_sq))
        simulate_node_length_with_parameters(G, root, simulated, alpha, beta, sigma_sq)

        for parent, child in G.edges:
            t = G[parent][child]['time']
            y = simulated[child]
            z = simulated[parent]
            rows.append({
                'Y': y,
                'Z': z,
                't': t
            })

    return pd.DataFrame(rows)

def estimate_alpha_beta_sigma2(data):
    X = data[['t', 'Z']]
    y = data['Y']

    weights = 1 / data['t']
    model = LinearRegression().fit(X, y, sample_weight=weights)
    alpha_hat = model.coef_[0]
    beta_hat = model.coef_[1]

    # Residual variance estimate of σ²

    residuals = y - model.predict(X)
    sigma_sq_hat = np.sum(weights * residuals**2) / len(y)

    return alpha_hat, beta_hat, sigma_sq_hat

root = 407
n = 1000
df = simulate_full_data(G, n, alpha=0.5, beta=1, sigma_sq=2500, alpha_0=50000, sigma_0_sq=5000)
alpha_hat, beta_hat, sigma_sq_hat = estimate_alpha_beta_sigma2(df)

print("Estimated alpha:", alpha_hat)
print("Estimated beta:", beta_hat)
print("Estimated sigma_sq:", sigma_sq_hat)

Estimated alpha: 0.5601814019849685
Estimated beta: 0.9999954093606939
Estimated sigma_sq: 2502.2942123815674


In [ ]:
def hard_assignment_EM(n, alpha_init, beta_init, sigma_sq_init, alpha_0_init, root=407):
    alpha = alpha_init
    beta = beta_init
    sigma_sq = sigma_sq_init
    alpha_0 = alpha_0_init

    for i in range(n):
        G = create_graph(tree, alpha, beta, sigma_sq)
        df = simulate_full_data(G, 100, alpha, beta, sigma_sq, alpha_0, 5000)
        single_dataset = df.iloc[:, 0]
        leaves = [node for node in G.nodes if G.out_degree(node) == 0]
        X_values = single_dataset[leaves]

        J_hat_z, h_hat_z = inference_algorithm(G, alpha, beta, sigma_sq, alpha_0, X_values, root)
        alpha_0_hat, _ = information_to_standard(J_hat_z, h_hat_z, root)
        alpha_hat, beta_hat, sigma_sq_hat = estimate_alpha_beta_sigma2(df)

        alpha = alpha_hat
        beta = beta_hat
        sigma_sq = sigma_sq_hat
        alpha_0 = alpha_0_hat
        if i % 4 == 0:
            print("Iteration:", i)
            print("Estimated alpha:", alpha)
            print("Estimated beta:", beta)
            print("Estimated sigma_sq:", sigma_sq)
            print("Estimated alpha_0:", alpha_0)  
        
    return alpha, beta, sigma_sq, alpha_0

hard_assignment_EM(50, 0.5, 1, 2500, 50000)

Iteration: 0
Estimated alpha: 0.5089949276253402
Estimated beta: 1.0000112235371617
Estimated sigma_sq: 2516.9592265481347
Estimated alpha_0: 50128.78403878674
Iteration: 4
Estimated alpha: 0.5803006964062555
Estimated beta: 1.0001149705577623
Estimated sigma_sq: 2496.7100961142537
Estimated alpha_0: 50104.94089359476
Iteration: 8
Estimated alpha: 0.6499466056609828
Estimated beta: 1.0000751787922177
Estimated sigma_sq: 2473.127344417362
Estimated alpha_0: 50010.56971030272
Iteration: 12
Estimated alpha: 0.6630091757137117
Estimated beta: 0.999970225726177
Estimated sigma_sq: 2466.151616416253
Estimated alpha_0: 49903.73638476002
Iteration: 16
Estimated alpha: 0.9023183025126851
Estimated beta: 0.999899136491708
Estimated sigma_sq: 2469.1180430915188
Estimated alpha_0: 49737.78239748085
Iteration: 20
Estimated alpha: 0.910995508855802
Estimated beta: 0.9998718594427254
Estimated sigma_sq: 2493.798962706359
Estimated alpha_0: 49536.32968339021
Iteration: 24
Estimated alpha: 0.9295327445

(1.1811298563617383, 0.9998123710231457, 2512.4431457027795, 48652.85007111289)